In [1]:
# training dataset
from datasets import load_dataset
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric, list_metrics


dataset = load_dataset("yelp_review_full")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
# 这个样例数据集不大，300M


/home/dltp_haonan/anaconda3/envs/llamaRelu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
metric=load_metric("accuracy")

/home/dltp_haonan/anaconda3/envs/llamaRelu/lib/python3.9/site-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [3]:
# import torch
# torch.cuda.is_available()
import sys 
# add transformer into path
sys.path.insert(0, '/disk3/Haonan/yanbo_random/bert_finetune/transformers/src')
import transformers
print(transformers.__file__)

from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import TrainingArguments
from transformers import TrainingArguments, Trainer
# from transformers import 
import numpy as np
import torch
ckpt_path = "/disk3/Haonan/yanbo_random/bert_finetune/checkpoint"

/home/dltp_haonan/anaconda3/envs/llamaRelu/lib/python3.9/site-packages/transformers/__init__.py


In [12]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
train_dataset.shape

(650000, 5)

In [2]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)
small_eval_dataset = eval_dataset.select(range(1000))
test_dataset = eval_dataset.select(range(1000,2000))
eval_dataset.shape

(50000, 5)

In [24]:
test_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 49000
})

In [32]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
training_args = TrainingArguments(output_dir=ckpt_path, evaluation_strategy="epoch")
training_args.learning_rate = 1e-5
training_args.num_train_epochs = 10
training_args.per_gpu_train_batch_size = 4
training_args._n_gpu = 2

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [14]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/home/dltp_haonan/anaconda3/envs/llamaRelu/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,1.003843
2,No log,0.972147
3,No log,0.953707
4,0.769300,0.974710
5,0.769300,0.983335


/home/dltp_haonan/anaconda3/envs/llamaRelu/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=625, training_loss=0.7037385681152344, metrics={'train_runtime': 209.8999, 'train_samples_per_second': 23.821, 'train_steps_per_second': 2.978, 'total_flos': 1315590712320000.0, 'train_loss': 0.7037385681152344, 'epoch': 5.0})

In [15]:
trainer.save_model(ckpt_path)

In [27]:
for batch in test_dataset:
    print(batch)
    break

{'label': 2, 'text': "Hefty portions, great value, homey comfortable service: not overly efficient, but not slow either. Stick to T-Bird specialties and typical fare. Being adventurous doesn't pay here. Ask the waiter what's cooking to find the best options. Pros: Cheap, hefty, homemade feel. Cons: distinct bar atmosphere, poor atypical fare.", 'input_ids': [101, 1124, 27944, 8924, 117, 1632, 2860, 117, 1313, 1183, 6062, 1555, 131, 1136, 16201, 7856, 117, 1133, 1136, 3345, 1719, 119, 24953, 1106, 157, 118, 8258, 1957, 4338, 1105, 4701, 14550, 119, 6819, 16889, 25576, 2144, 112, 189, 2653, 1303, 119, 18149, 1103, 17989, 1184, 112, 188, 8739, 1106, 1525, 1103, 1436, 6665, 119, 5096, 1116, 131, 20394, 4490, 1643, 117, 1119, 27944, 117, 1313, 12540, 1631, 119, 16752, 1116, 131, 4966, 2927, 6814, 117, 2869, 1120, 1183, 15328, 14550, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [30]:
# test with origin dataset
model = AutoModelForSequenceClassification.from_pretrained("/disk3/Haonan/yanbo_random/bert_finetune/checkpoint/checkpoint-500",num_labels=5)
model.eval()
metric=load_metric("accuracy")
predictions = []
with torch.no_grad():
    step = 0
    for batch in test_dataset:
        if step %100 == 0:
            print(step)
        step +=1
        # add batch size?
        inputs = {k: torch.tensor(v) for k, v in batch.items() if k in tokenizer.model_input_names}
        # print(inputs)
        for k in inputs.keys():
            inputs[k]= inputs[k].unsqueeze(0)
        outputs = model(**inputs)
        # print(outputs)
        prediction = torch.tensor(torch.argmax(outputs.logits, dim=-1))
        predictions.append(prediction)
    metric.add_batch(predictions=predictions, references=test_dataset["label"])
score = metric.compute() 
print(score)

/home/dltp_haonan/anaconda3/envs/llamaRelu/lib/python3.9/site-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


0


/tmp/ipykernel_580855/2941891565.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prediction = torch.tensor(torch.argmax(outputs.logits, dim=-1))


100
200
300
400
500
600
700
800
900
{'accuracy': 0.547}


In [5]:
# 用glue来测试
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli_mismatched": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

test_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
datasets = {}
def test_tokenize_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return test_tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)
for td in task_to_keys.keys():
    sentence1_key, sentence2_key = task_to_keys[td]
    dataset = load_dataset('glue',td)
    test_tokenized_datasets = dataset.map(test_tokenize_function, batched=True)
    test_tokenized_datasets.shape
    datasets[td] = test_tokenized_datasets

Map: 100%|██████████| 67349/67349 [00:01<00:00, 37916.45 examples/s]


In [5]:
datasets

{'cola': DatasetDict({
     train: Dataset({
         features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
         num_rows: 8551
     })
     validation: Dataset({
         features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
         num_rows: 1043
     })
     test: Dataset({
         features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
         num_rows: 1063
     })
 }),
 'mnli': DatasetDict({
     train: Dataset({
         features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
         num_rows: 392702
     })
     validation_matched: Dataset({
         features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
         num_rows: 9815
     })
     validation_mismatched: Dataset({
         features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mas

In [6]:
# datasets["cola"]['test'].shape#
result = {}
for task_name, task_dataset in datasets.items():
    num_labels = 3 if task_name =='mnli' else 2
    if task_name =='stsb': num_labels = 1
    # test
    model = AutoModelForSequenceClassification.from_pretrained("/disk3/Haonan/yanbo_random/bert_finetune/checkpoint/checkpoint-500",num_labels=5)
    model.eval()
    metric = load_metric('glue', task_name)
    eval_dataset = task_dataset["validation_matched" if task_name == "mnli" else "validation"]
    
    with torch.no_grad():
        for batch in eval_dataset:
            # add batch size?
            inputs = {k: torch.tensor(v) for k, v in batch.items() if k in test_tokenizer.model_input_names}
            # print(inputs)
            for k in inputs.keys():
                inputs[k]= inputs[k].unsqueeze(0)
            outputs = model(**inputs)
            # print(outputs)
            predictions = torch.argmax(outputs.logits, dim=-1)
            print(predictions)
            # metric.add_batch(predictions=predictions, references=batch["label"])
    score = metric.compute() 
    result[task_name] = score
print(result)

/tmp/ipykernel_580855/3993239749.py:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', task_name)
/home/dltp_haonan/anaconda3/envs/llamaRelu/lib/python3.9/site-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


tensor([3])


TypeError: 'int' object is not iterable

In [17]:
task_name = 'cola'
task_dataset = datasets['cola']
    # num_labels = 3 if task_name =='mnli' else 2
    # if task_name =='stsb': num_labels = 1
# test
model = AutoModelForSequenceClassification.from_pretrained("/disk3/Haonan/yanbo_random/bert_finetune/checkpoint/checkpoint-500",num_labels=5)
model.eval()
metric = load_metric('glue', task_name)
eval_dataset = task_dataset["validation_matched" if task_name == "mnli" else "validation"]
print(eval_dataset)
inputs = {}
# print(test_tokenizer.model_input_names)
for batch in eval_dataset:
    inputs = {k: torch.tensor(v) for k, v in batch.items() if k in test_tokenizer.model_input_names}
    # for k,v in batch.items():
    #     if k in test_tokenizer.model_input_names:
    #         inputs[k]=torch.tensor(v)
    for k in inputs.keys():
        inputs[k]= inputs[k].unsqueeze(0)
    print(inputs)
# with torch.no_grad():
#     for batch in eval_dataset:
#         # add batch size?
#         inputs = {k: torch.tensor(v) for k, v in batch.items() if k in test_tokenizer.model_input_names}

/home/dltp_haonan/anaconda3/envs/llamaRelu/lib/python3.9/site-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Dataset({
    features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1043
})
{'input_ids': tensor([[  101,  1109, 12428,  8335,  1103, 10647,  2330,  1104,  1103,  5753,
           119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[  101,  1109, 17981,  1189,  1103,  8090,  7461,  1166,  1103,  3373,
          2254,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[  101,  1109,  6676, 13559,   192, 17305,  8384,  2111,  5768,   119,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[ 101, 1409, 1128, 1125, 8527, 1167,  117, 1128, 1156, 1328, 1750,  119,
          102]]), 'token_ty